In [1]:
!gdown --id 1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU
To: /kaggle/working/denver_extract.mp3
100%|██████████████████████████████████████| 14.4M/14.4M [00:00<00:00, 66.3MB/s]


In [ ]:
# 

# step 1: Speech to Text

In [3]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [1]:
from transformers import pipeline
import torch

# load audio
audio_filename = "denver_extract.mp3"

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium.en",
    device = "cuda",
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

 kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So, thank you. Thank you so much and thanks for your leadership. All right, welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the 

# step 2: Analysis

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextStreamer,
    BitsAndBytesConfig
)

QWEN_MODEL = "Qwen/Qwen2.5-7B-Instruct"

system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


# quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    QWEN_MODEL,
    trust_remote_code=True
)

# Qwen DOES have a pad token, but this is still a safe
tokenizer.pad_token = tokenizer.eos_token

# apply chat template
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt"
).to("cuda")

# streamer
streamer = TextStreamer(tokenizer)


# model
model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)


# generation
outputs = model.generate(
    inputs,
    max_new_tokens=2000,
    streamer=streamer
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
<|im_end|>
<|im_start|>user

Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence 

In [12]:
response = tokenizer.decode(outputs[0])
response

"<|im_start|>system\n\nYou produce minutes of meetings from transcripts, with summary, key discussion points,\ntakeaways and action items with owners, in markdown format without code blocks.\n<|im_end|>\n<|im_start|>user\n\nBelow is an extract transcript of a Denver council meeting.\nPlease write minutes in markdown without code blocks, including:\n- a summary with attendees, location and date\n- discussion points\n- takeaways\n- action items with owners\n\nTranscription:\n kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about

## minimal version

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1) Choose a small Qwen instruct model
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

# 2) Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# 3) Load model (no quantization, no tricks)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cuda", torch_dtype=torch.float16)


# 4) Define a VERY simple prompt
prompt = "write a sentence about what meeting agenda is"

# 5) Tokenize (text → numbers)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 6) Generate
outputs = model.generate(**inputs, max_new_tokens=200 )

# 7) Decode (numbers → text)
result = tokenizer.decode(outputs[0])
print(result)

write a sentence about what meeting agenda is. A meeting agenda serves as a roadmap that outlines the topics, decisions, and actions to be discussed or taken during the meeting, ensuring everyone is on the same page and productive in their contributions.

This sentence provides a clear definition of what a meeting agenda is: it's a document or list that specifies the details of an upcoming meeting, including the main points to be covered, decisions to be made, and tasks to be accomplished. It helps ensure that all participants know what they're expected to contribute and how their input will be used.
To expand further:

A meeting agenda acts as a guide for discussion, decision-making, and action planning. It typically includes sections such as "Agenda," "Discussion Points," "Decision Items," and "Action Items." The "Agenda" section lists each item that will be discussed at the meeting; the "Discussion Points" section highlights any key questions or areas where more information may be n